In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from joblib import dump, load
import json

## Preprocessing

In [ ]:
data=pd.read_csv("dataset.csv")

## Training

## Exporting Modle

## Inferencing